In [ ]:
import pandas as pd
import altair as alt
from umap import UMAP
import numpy as np
import geopandas as gpd

alt.data_transformers.disable_max_rows()
SEED = 42

In [ ]:
# first objective for a modeling step is to identify positive and negative training examples
# 2 sources: reachwell competitors and reachwell clients

In [ ]:
# 1. limited competitor list based on a review of each competitor's website and press:

parent_square = [
    "Oakland Unified School District",      "9700000US0628050",
    "Virginia Beach City Public Schools",   "9700000US5103840",
    "Knox County Schools",                  "9700000US4702220", # (Tennessee)
    "Fort Wayne Community Schools",         "9700000US1803630",
    "Beaverton School District",            "9700000US4101920",
    "Andrews Independent School District",  "9700000US4808280",
]

bloomz = [
    "Johnston County Public Schools",               "9700000US3702370",
    "Live Oak School District",                     "9700000US0622050",
    "Fort Sam Houston Independent School District", "9700000US4820160",
    "Windsor Unified School District",              "9700000US0600034",
    "Licking Heights Local Schools",                "9700000US3904800",
    "Ozark City Schools",                           "9700000US0102640",
    "Henderson County Public Schools",              "9700000US2102710",
    # https://kipptexas.org/ -- these schools are located in Houston, Dallas, San Antonio, Austin
    "KIPP Texas Public Schools",                    "9700000US4823640", # Houston
    "KIPP Texas Public Schools",                    "9700000US4816230", # Dallas
    "KIPP Texas Public Schools",                    "9700000US4838730", # San antonio
    "KIPP Texas Public Schools",                    "9700000US4808940", # Austin
    "North Lawrance Community Schools",             "9700000US1807860",
    "Round Rock Independent School District",       "9700000US4838080",
    "NYC Department of Education",                  "9700000US3620580",
    "Los Angeles Unified School District",          "9700000US0622710"
]

talking_points = [ 
    "Wake County Public Schools",        "9700000US3704720",
    "Boston Public Schools",             "9700000US2502790",
    "Cincinnati Public Schools",         "9700000US3904375",
    "Seattle Public Schools",            "9700000US5307710",
    "Elk Grove Unified School District", "9700000US0612330",
    "Aurora Public Schools",             "9700000US3103360",
    "Tulsa Public Schools",              "9700000US4030240",
    "Pittsburgh Public Schools",         "9700000US4024180"
]

competitors = []
for cname,ids in [ ("TalkingPoints",talking_points), ("Bloomz", bloomz), ("ParentSquare", parent_square) ]:  
    competitors += [ 
        {"geo_id":id_, "uses_competitor": cname} for id_ in ids[1::2]
    ]

competitors_df = pd.DataFrame.from_records(competitors)

competitors_df

In [ ]:
# 2. reachwell's client list, which was provided to me as a .kml file of coordinate points alongside the client name. I converted these to school district and county 
# IDs by finding the school district corresponding to each geographic point. fortunately this is implemented in geopandas. see `extract_client_list.ipynb` for details.
reachwell_df = pd.read_csv("../reachwell-clients-with-counties.csv")
reachwell_df["label"] = 1
reachwell_df["is_reachwell_partner"] = 1

In [ ]:
# next step is acquiring features: 
# merge indicators with labeled data and transform some features

indicators = pd.read_csv("../cty-sd-latest-indicators.csv")
indicators = indicators.merge(reachwell_df,how="left",on="geo_id")
indicators = indicators.merge(competitors_df,how="left",on="geo_id")

indicators.loc[~indicators["uses_competitor"].isna(), "label"] = 1

indicators["uses_competitor"] = indicators["uses_competitor"].fillna("")
indicators["label"] = indicators["label"].fillna(0)
indicators["is_reachwell_partner"] = indicators["is_reachwell_partner"].fillna(0)

indicators = indicators.dropna().reset_index(drop=True)
indicators["label"].value_counts()

indicators["log_number_households"] = indicators["total_households"].apply(np.log)
indicators["log_median_household_income"] = indicators["median_income_households_with_children"].apply(np.log)
indicators["state"] = indicators["geo_name"].apply(lambda s: s.split(", ")[-1])
# neat. now we have a very imbalanced classification problem

In [ ]:
# next, try to better understand this data. particularly, are reachwell clients similar to eachother? what cohorts exists within the data? 
# do this with dimensionality reduction because there are a lot of variables

reducer = UMAP(n_neighbors=9)
percent_cols = [
       "pct_limited_english_households","pct_pop_below_poverty_level","pct_speaks_other_than_english","pct_households_with_chilren",
       "pct_households_receiving_food_stamps", "pct_households_with_children_married_family", "pct_pop_3+_in_private_school"
]

umap_df = pd.DataFrame(
       reducer.fit_transform(indicators[percent_cols]),
       columns=['UMAP1', 'UMAP2']
)

In [ ]:
result_df = pd.concat([indicators, umap_df], axis=1)

result_df["log_total_households"] = result_df["total_households"].apply(np.log)

alt.Chart(result_df).mark_circle(size=75,opacity=0.6,stroke='black',strokeWidth=0.4).encode(
    x='UMAP1:Q',
    y='UMAP2:Q',
    color='label:N',
    # size=alt.Size('total_households:Q',legend=None),
    tooltip=['geo_id', 'geo_name', 'total_households:Q','pct_limited_english_households','pct_pop_below_poverty_level']
).properties(
    width=800,
    height=800
).interactive()


In [ ]:

# set up a classification problem using `good` as attributes/features (all 0-1 normalized indicators that are not colinear with eachother, as well as log population size)
feature_cols = percent_cols + ["log_number_households","log_median_household_income"]

# what are good metrics for this learning problem?

# precision very important. Recall also important but less so (for example i could return the whole list of scholl districts and counties and
# this would yield a recall of 1.0... not so helpful). because this will inevitably be a ranking problem (ordering a list of leads by how 
# similar they are to existing clients), I can use ranking metrics.
# a good selection would be MAP@K (or really just AP b/c only ranking one 'query') as there are multiple true values in the impression list but their order is not important


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, precision_score, accuracy_score, balanced_accuracy_score, recall_score,f1_score, average_precision_score
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC


In [ ]:
train__imbalanced, test = train_test_split(indicators, train_size=0.8, random_state=SEED,stratify=indicators["label"]) # retrieve 

In [ ]:

negative_samples = 100
train = pd.concat([
    train__imbalanced[train__imbalanced.label == 1], 
    train__imbalanced[train__imbalanced.label == 0].sample(n=negative_samples, replace=False, random_state=SEED)
]).sample(frac=1).reset_index(drop=True)


In [ ]:
train["label"].value_counts(), train["geo_id"].nunique()

In [ ]:
def evaluate_model(clf, df_: pd.DataFrame, k=500): #-> Dict[str,float]:

    
    # run this on the imbalanced versiosn of train and test set
    df = df_.copy(deep=True).reset_index(drop=True)
    df["prediction"] = clf.predict(df[feature_cols])
    df["ranking"] = clf.predict_proba(df[feature_cols])[:,1]

    gt = set(df[df.label==1]["geo_id"].tolist())
    top_k = set(df.sort_values(by="ranking",ascending=False).head(k)["geo_id"].tolist())
    return {
        "balanced_accuracy":    balanced_accuracy_score(y_true=df["label"],y_pred=df["prediction"]),
        "precision":            precision_score(y_true=df["label"],y_pred=df["prediction"],zero_division=0),
        "recall":               recall_score(y_true=df["label"],y_pred=df["prediction"],zero_division=0),
        "f1":                   f1_score(y_true=df["label"],y_pred=df["prediction"],zero_division=0),

        # ranking metrics: web.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
        "average_precision": average_precision_score(y_true=df["label"],y_score=df["ranking"]),
        "precision_at_k":    len(gt & top_k)/k, # what percent of results in the first 500 are relevant
        # "recall_at_k":       len(gt & top_k)/len(gt) # what percent of results in the first 500 are relevant
    }

In [ ]:

param_grid_lr = {
    'penalty': ['l2'],  
    'C': [0.001, 0.01, 0.03, 0.1, 1, 10],  
    'class_weight': [{0:1,1:3},{0:1,1:1}]
}


gs_lr = GridSearchCV(
    estimator=LogisticRegression(solver="liblinear"),
    param_grid=param_grid_lr,
    # scoring=make_scorer(accuracy_score),
    scoring=make_scorer(precision_score,zero_division=0),
    cv=10
)
gs_lr.fit(train[feature_cols],train["label"])


param_grid_knn = {
    "n_neighbors": [3,5,7,9],
    "weights": ["distance","uniform"]
}


gs_knn = GridSearchCV(
    estimator=KNeighborsClassifier(),
    param_grid=param_grid_knn,
    # scoring=make_scorer(accuracy_score),
    scoring=make_scorer(precision_score,zero_division=0),
    cv=10
)
gs_knn.fit(train[feature_cols],train["label"])

param_grid_svc = {
    'C': [1],#[0.001, 0.01, 0.03, 0.1, 1, 10],  
    "kernel":["rbf","poly"],
    "degree":[2], # [1,2]
    "class_weight":["balanced",None]
}

gs_kernel = GridSearchCV(
    estimator=SVC(probability=True),
    param_grid=param_grid_svc,
    scoring=make_scorer(precision_score,zero_division=0),
    # scoring=make_scorer(accuracy_score),
    cv=10
)

gs_kernel.fit(train[feature_cols],train["label"])

In [ ]:
# baseline is a dummy model
dummy = GridSearchCV(
    estimator=DummyClassifier(),
    param_grid={"strategy":["constant"],"constant":[1]},
    # scoring=make_scorer(precision_score,zero_division=0),
    scoring=make_scorer(accuracy_score),
    cv=10
)
dummy.fit(train[feature_cols],train["label"])



In [ ]:
# pd.DataFrame(gs_lr.cv_results_).sort_values("mean_test_score", ascending=False)
# pd.DataFrame(gs_knn.cv_results_).sort_values("mean_test_score",ascending=False)
# pd.DataFrame(d.cv_results_)

from itertools import product
experiment_results = []
for (name,model),dataset in product(
        {"dummy": dummy, "logistic regression": gs_lr, "k nearest neighors":gs_knn,"support vecter machine":gs_kernel}.items(),
        ["test"]
):
    d = {"model":name,"dataset":dataset}
    # print(name,model,dataset)
    # break
    # prit(model)
    r = evaluate_model( model, test if dataset == "test" else train__imbalanced,k=500 )

    experiment_results.append(d | r)



In [ ]:
pd.DataFrame.from_records(experiment_results)

In [ ]:
indicators["prediction"] = gs_lr.predict_proba(indicators[feature_cols])[:,1]
plot_df = indicators[indicators.geo_type == "CTY"].copy(deep=True)
plot_df["id"] =  plot_df["geo_id"].apply(lambda s: int(s.split("US")[1]))


In [ ]:
# model weights
# sorted(zip(list(list(gs_lr.best_estimator_.coef_[0,:])),feature_cols))

In [ ]:
import altair as alt
from vega_datasets import data

counties = alt.topo_feature(data.us_10m.url, 'counties')

alt.Chart(counties).mark_geoshape().encode(
    color=alt.Color('prediction:Q',scale=alt.Scale(scheme="inferno")),
    tooltip=["geo_name:N","prediction:Q","pct_speaks_other_than_english:Q","median_income_households_with_children:Q","total_households:Q"]
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(plot_df, 'id', ['prediction','geo_name','pct_speaks_other_than_english',"median_income_households_with_children","total_households"])
).project(
    type='albersUsa'
).properties(
    width=1200,
    height=900
)

In [ ]:
# output the df (with predictions) as an excel file
indicators.sort_values(by="prediction", ascending=False).to_excel("../outputs/school-districts-counties-predictions.xlsx")

In [ ]:

# and the geojson file for plotting out counties with deck.gl and altair
gdf_counties = gpd.read_file("../data/cty-geo/geojson-counties-fips.json")


cols_for_plot = [
    "geo_id","geo_name", "geometry", "total_households", "total_population_5+", "median_income_households_with_children", "pct_limited_english_households", "uses_competitor",
    "pct_pop_below_poverty_level", "pct_households_with_chilren", "pct_speaks_only_english", "pct_speaks_other_than_english",
    "pct_households_receiving_food_stamps", "pct_households_with_children_married_family", "pct_pop_3+_in_private_school", "pct_pop_3+_in_public_school",
]
indcators[cols_for_plot].to_csv("../outputs/county-plots.csv",index=Fals)
cols_for_private_plot = cols_for_plot + ["prediction"]

gdf_counties = gdf_counties.merge(indicators[indicators.geo_type == "CTY"], left_on="GEO_ID", right_on="geo_id", how="inner") 


In [ ]:
gdf_counties[cols_for_plot].to_file("../outputs/county-plots.json",driver="GeoJSON")
gdf_counties[cols_for_private_plot].to_file("../outputs/county-plots-private.json",driver="GeoJSON")